In [2]:
#Loading Data
import pandas as pd
data = pd.read_csv("Churn_Modelling.csv")
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [3]:
# Dropping unwanted Columns
data = data.drop(["RowNumber" , "Surname" , "CustomerId"] , axis = 1)
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [6]:
#Converting Gender with labelEncoder
from sklearn.preprocessing import LabelEncoder
genderLabelEncoder = LabelEncoder()
data['Gender'] = genderLabelEncoder.fit_transform(data['Gender'])

In [7]:
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0


In [19]:
from sklearn.preprocessing import OneHotEncoder
geographyEncoder = OneHotEncoder()
encoder_geo = geographyEncoder.fit_transform(data[['Geography']])


In [20]:
geographyEncoder.get_feature_names_out(['Geography'])

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [21]:
geoEncodedData = pd.DataFrame(encoder_geo.toarray(), columns = geographyEncoder.get_feature_names_out(['Geography']))

In [23]:
geoEncodedData

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [25]:
data = pd.concat([data.drop(['Geography'] , axis = 1) , geoEncodedData] , axis = 1)

In [26]:
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [40]:
from sklearn.model_selection import train_test_split

X = data.drop('EstimatedSalary' , axis = 1)
y = data['EstimatedSalary']

X_train, X_test , y_train, y_test = train_test_split(X,y,train_size=0.8,random_state=10)


In [42]:
from sklearn.preprocessing import StandardScaler
scalar = StandardScaler()
X_train = scalar.fit_transform(X_train)
X_test = scalar.transform(X_test)

In [46]:
import tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense


model = Sequential([
    Dense(64,activation = "relu" , input_shape = (X_train.shape[1],)),
    Dense(32,activation= "relu"),
    Dense(1),
])

/opt/anaconda3/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [47]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,945 (11.50 KB)

 Trainable params: 2,945 (11.50 KB)

 Non-trainable params: 0 (0.00 B)

In [50]:
import pickle

with open("Regression_scalar.pkl", "wb") as file:
    pickle.dump(scalar, file)

with open("Regression_gender_encoder.pkl" , "wb") as file:
    pickle.dump(genderLabelEncoder,file)

with open("Regression_Geography_Encoder.pkl" , "wb") as file:
    pickle.dump(geographyEncoder , file)


In [51]:
import tensorflow
opt = tensorflow.keras.optimizers.Adam(learning_rate= 0.01)


In [60]:
from tensorflow.keras.callbacks import TensorBoard , EarlyStopping
import datetime

model.compile(optimizer = opt , loss = 'mean_absolute_error' , metrics = ['mae'])

log_dir = "regressionLogs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

tensorboard_callback = TensorBoard(log_dir=log_dir, profile_batch=5)
earlystopping_callback = EarlyStopping(restore_best_weights = True , patience = 5 , monitor="val_loss")

model.fit(X_train , y_train , validation_data= (X_test , y_test) , epochs = 100 , callbacks = [tensorboard_callback , earlystopping_callback])

Epoch 1/100


2025-03-30 15:55:51.857587: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:103] Profiler session initializing.
2025-03-30 15:55:51.858109: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:118] Profiler session started.
2025-03-30 15:55:51.859113: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:130] Profiler session tear down.


152/250 ━━━━━━━━━━━━━━━━━━━━ 0s 665us/step - loss: 48642.1406 - mae: 48642.1406 

2025-03-30 15:55:52.276903: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:103] Profiler session initializing.
2025-03-30 15:55:52.276916: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:118] Profiler session started.
2025-03-30 15:55:52.301167: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:68] Profiler session collecting data.
2025-03-30 15:55:52.304084: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:130] Profiler session tear down.
2025-03-30 15:55:52.305833: I external/local_xla/xla/tsl/profiler/rpc/client/save_profile.cc:147] Collecting XSpace to repository: regressionLogs/fit/20250330-155551/train/plugins/profile/2025_03_30_15_55_52/Manjunadhs-MacBook-Air.local.xplane.pb


250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 937us/step - loss: 48738.1211 - mae: 48738.1211 - val_loss: 50226.1875 - val_mae: 50226.1875
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 546us/step - loss: 48367.5312 - mae: 48367.5312 - val_loss: 50327.0312 - val_mae: 50327.0312
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 552us/step - loss: 49202.5898 - mae: 49202.5898 - val_loss: 50364.6289 - val_mae: 50364.6289
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 533us/step - loss: 48583.2539 - mae: 48583.2539 - val_loss: 50210.9922 - val_mae: 50210.9922
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 528us/step - loss: 49090.1133 - mae: 49090.1133 - val_loss: 50265.1719 - val_mae: 50265.1719
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 829us/step - loss: 49217.5469 - mae: 49217.5469 - val_loss: 50297.0703 - val_mae: 50297.0703
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 622us/step - loss: 48600.9883 - mae: 48600.9883 - val_loss: 50311.9336 - val_mae: 50311.9336
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 618us/ste

In [62]:
%load_ext tensorboard
%tensorboard --logdir regressionLogs/fit



The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6006 (pid 49331), started 0:05:40 ago. (Use '!kill 49331' to kill it.)

In [64]:
import pickle
with open("Regression_model.pkl" , "wb") as file:
    pickle.dump(scalar,file)


In [65]:
model.save("Regression_model.h5")